## Activation Function(활성화 함수)
- 활성화 함수는 네트워크를 통과해온 값을 최종적으로 어떤 값으로 만들지를 결정
- 비선형 함수이면서 미분가능한 함수여야함  
    선형일 경우, 네트워크 안의 선형적인 계산과 별반 다를 것이 없게 됨(즉, 쓸모가 없음)  
    backpropagation시 미분 값을 전파함으로 미분이 가능한 함수여야 함  
- Sigmoid, tanh, ReLU등이 있음 (최근에는 ReLU를 많이 씀)

## Backporpagaiton(역전파)
- 네트워크를 따라서 나온 결과값과 정답과의 오차를 신경망을 따라 입력층까지 역으로 전파하여 계산
- 미분값을 전해주며, chain rule에 따라 모든 값을 곱해나감

In [1]:
import numpy as np
import tensorflow as tf

#### 1. Data Preprocessing(데이터 준비하기)
1. 털과 날개가 있느냐를 담은 특징 데이터를 준비  
   있으면 1, 없으면 0

In [2]:
# [털, 날개]
x_data = np.array([[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

2. 각 객체가 어떤 종류인지를 알려주는 레이블 데이터(정답값)  
    one-hot-encoding(원핫인코딩)을 사용  
    (원핫인코딩은 다음과 같이 해당되는 부분만을 1로 표현하는 표기법)
    - 기타 = [1, 0, 0]
    - 포유류 = [0, 1, 0]
    - 조류 = [0, 0, 1]  


In [3]:
y_data = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

#### 2. Model(신경망 구성하기)

In [4]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [6]:
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

In [7]:
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L) #activation func인 relu 사용

In [8]:
# softmax함수를 이용하여 확률값처럼 이용할 수 있게 만들어줌  
# (softmax함수를 통과하면 모든 값의 합이 1이되므로 확률처럼 해석할 수 있음)
model = tf.nn.softmax(L)

In [12]:
#loss func으로는 cross-entropy(교차 엔트로피)를 사용(보통 원핫인코딩시에 많이 사용함)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis = 1))

In [13]:
#optimizer = GD
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

#### 3. Training (학습하기)

In [24]:
#session 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#training = 100번
for step in range(100) :
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0 :
        print(step+1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 0.9710645
20 0.9666026
30 0.96228045
40 0.9580889
50 0.95403546
60 0.9501911
70 0.9463763
80 0.9427941
90 0.9392418
100 0.93581086


#### 4. Evaluation(모델 평가하기)

In [25]:
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print("예측값:", sess.run(prediction, feed_dict={X: x_data}))
print("정답값:", sess.run(target, feed_dict={Y: y_data}))

예측값: [0 2 2 0 0 2]
정답값: [0 1 2 0 0 2]


In [26]:
#정확도 출력
is_correct = tf.equal(prediction, target) #tf.equal을 통해 예측값과 정답값을 비교 (true, false)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32)) #tf.cast를 통해 0과 1로 바꿈
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 83.33
